Import libraries

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.kernel_approximation import RBFSampler
from sklearn.model_selection import train_test_split 
import joblib
import keras
from keras.models import Model
from keras.layers import Input, Dense
from sklearn.svm import OneClassSVM
from keras.optimizers import Adam
from sklearn.metrics import  precision_score, recall_score, f1_score, average_precision_score, precision_recall_curve,accuracy_score, roc_auc_score

Training dataset in deep kernel one class classification model

In [ ]:
training_dataset = pd.read_csv('Complete_training_heart.csv')


X = training_dataset.drop('Normal heart', axis=1)
y = training_dataset.drop([' PULSE', ' SpO2'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, shuffle=True,random_state=42)

'''# Scale the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)
y_scaled = scaler.fit_transform(y_train)
X_test_scaled = scaler.fit_transform(X_test)'''

# Define the deep neural network architecture
inputs = Input(shape=2, name='input')
hidden1 = Dense(30, activation='relu',name='dense_1')(inputs)
hidden2 = Dense(15, activation='relu',name='dense_2')(hidden1)
hidden3 = Dense(10, activation='relu',name='dense_3')(hidden2)
hidden4 = Dense(5, activation='relu',name='dense_4')(hidden3)
outputs = Dense(1, activation='sigmoid',name='dense_5')(hidden4)

# Create the model
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics= ['accuracy'])
model.fit(X_train, y_train, epochs= 20, batch_size=32)

# Extract features from the hidden layer
feature_model = Model(inputs=model.inputs, outputs=model.get_layer('dense_1').output)
X_train_features = feature_model.predict(X_train)
X_test_features = feature_model.predict(X_test)

# Apply a kernel function to the extracted features
rbf = RBFSampler(gamma=0.1, n_components= 2)  
X_train_rbf = rbf.fit_transform(X_train_features)
X_test_rbf = rbf.transform(X_test_features)

# Train a one-class SVM on the kernel-transformed features
ocsvm_rbf = OneClassSVM(kernel='rbf', nu=0.1)  
ocsvm_rbf.fit(X_train_rbf)

# Predict the labels for the input data
y_pred_ocsvm_rbf = ocsvm_rbf.predict(X_test_rbf)
y_pred_ocsvm_rbf = np.where(y_pred_ocsvm_rbf == -1, 0, 1)

positive_percentage = (y_test).sum() / len(y_pred_ocsvm_rbf)

# Compute evaluation metrics
precision = precision_score(y_test, y_pred_ocsvm_rbf)
recall = recall_score(y_test, y_pred_ocsvm_rbf)
f1 = f1_score(y_test, y_pred_ocsvm_rbf)


outlier_count = (y_pred_ocsvm_rbf == -1).sum()
print ("Positive percentage:", positive_percentage)
print("Outlier count:", outlier_count)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)  
#print("Accuracy ", scores.mean())

#precision, recall, thresholds = precision_recall_curve(y, y_pred_ocsvm_rbf)
# Calculate AUPRC
auprc = average_precision_score (y_test, y_pred_ocsvm_rbf)
print("AUPRC:", auprc)
# Print the predicted labels
print("Deep Kernel-Based Predictions:", y_pred_ocsvm_rbf)

filename = 'Heart_attack_monitor_DNN-Kernel.sav'
joblib.dump(ocsvm_rbf, filename)


Testing the deep kernel one class classification performance 

In [4]:
# Load a model
DNNkernel = joblib.load('Heart_attack_monitor_DNN-Kernel.sav')

val_dataset = pd.read_csv('Validate_heart.csv')
x_val = val_dataset.drop('Normal heart', axis=1)

y_val = val_dataset['Normal heart']  
y_val_features = np.reshape(y_val, (-1, 1))

y_pred = DNNkernel.predict(x_val)
y_pred = np.where(y_pred == -1, -1, 1)

positive_percentage = (y_val).sum() / len(y_pred)
precision, recall, thresholds = precision_recall_curve(y_val, y_pred)
'''
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
'''

# Compute evaluation metrics
f1 = f1_score(y_val, y_pred)

roc_auc = roc_auc_score(y_val, y_pred)

# Print the evaluation scores

outlier_count = (y_val == -1).sum()

print ("Positive percentage:", positive_percentage)
print("Outlier count:", outlier_count)
print("Precision:", precision)
print("Recall:", recall)
print("Threshold:", thresholds)
print("ROC AUC", roc_auc)
print("F1-score:", f1)  
#print("Accuracy ", scores.mean())

# Calculate AUPRC
auprc = average_precision_score(y_val, y_pred)
print("AUPRC:", auprc)

c:\Users\jessl\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but OneClassSVM was fitted without feature names
  warnings.warn(


Positive percentage: 0.9599679743795037
Outlier count: 500
Precision: [0.97998399 1.        ]
Recall: [1. 0.]
Threshold: [-1]
ROC AUC 0.5
F1-score: 0.0
AUPRC: 0.9799839871897518
